# YC1

In [3]:
from constraint import Problem

# 5x5 board, 5 queens
N = 5

# 1. Define problem
problem = Problem()

# 2. Variables:
#    One variable per row, value = column index of queen in that row
rows = range(N)
columns = range(N)
problem.addVariables(rows, columns)

# 3. Constraints:
#    (a) All queens must be in different columns
problem.addConstraint(lambda *cols: len(set(cols)) == N, rows)

#    (b) No two queens on same diagonal
#        For any two different rows r1, r2:
def no_diagonal_conflict(c1, c2, r1, r2):
    return abs(c1 - c2) != abs(r1 - r2)

for r1 in rows:
    for r2 in rows:
        if r1 < r2:
            problem.addConstraint(
                lambda c1, c2, r1=r1, r2=r2: no_diagonal_conflict(c1, c2, r1, r2),
                (r1, r2)
            )

# 4. Solve
solutions = problem.getSolutions()

print(f"Number of solutions: {len(solutions)}")
for s in solutions:
    print(s)

# Optional: pretty print a board for each solution
def print_board(solution):
    for r in range(N):
        line = ""
        for c in range(N):
            line += "Q " if solution[r] == c else ". "
        print(line)
    print()

for sol in solutions:
    print_board(sol)

Number of solutions: 10
{0: 4, 1: 2, 2: 0, 3: 3, 4: 1}
{0: 4, 1: 1, 2: 3, 3: 0, 4: 2}
{0: 3, 1: 1, 2: 4, 3: 2, 4: 0}
{0: 3, 1: 0, 3: 4, 2: 2, 4: 1}
{0: 2, 1: 4, 2: 1, 3: 3, 4: 0}
{0: 2, 1: 0, 2: 3, 3: 1, 4: 4}
{0: 1, 1: 3, 2: 0, 3: 2, 4: 4}
{0: 1, 1: 4, 3: 0, 2: 2, 4: 3}
{0: 0, 1: 2, 2: 4, 3: 1, 4: 3}
{0: 0, 1: 3, 2: 1, 3: 4, 4: 2}
. . . . Q 
. . Q . . 
Q . . . . 
. . . Q . 
. Q . . . 

. . . . Q 
. Q . . . 
. . . Q . 
Q . . . . 
. . Q . . 

. . . Q . 
. Q . . . 
. . . . Q 
. . Q . . 
Q . . . . 

. . . Q . 
Q . . . . 
. . Q . . 
. . . . Q 
. Q . . . 

. . Q . . 
. . . . Q 
. Q . . . 
. . . Q . 
Q . . . . 

. . Q . . 
Q . . . . 
. . . Q . 
. Q . . . 
. . . . Q 

. Q . . . 
. . . Q . 
Q . . . . 
. . Q . . 
. . . . Q 

. Q . . . 
. . . . Q 
. . Q . . 
Q . . . . 
. . . Q . 

Q . . . . 
. . Q . . 
. . . . Q 
. Q . . . 
. . . Q . 

Q . . . . 
. . . Q . 
. Q . . . 
. . . . Q 
. . Q . . 



# YC2

In [4]:
import time

N = 5  # số hậu (chỉ 5 quân hậu)

# --- Hàm kiểm tra ràng buộc ---
def is_safe(solution, row, col):
    for r, c in enumerate(solution):
        if c == col or abs(c - col) == abs(r - row):
            return False
    return True

# --- Hàm in bàn cờ ---
def print_board(solution):
    for r in range(N):
        row = ""
        for c in range(N):
            if r < len(solution) and solution[r] == c:
                row += " Q "
            else:
                row += " . "
        print(row)
    print("-" * (3 * N))

# --- Chiến lược 1: Backtracking cơ bản (chọn hàng theo thứ tự) ---
def backtrack_basic(row=0, solution=None, solutions=None):
    if solution is None:
        solution, solutions = [], []
    if row == N:
        solutions.append(solution[:])
        return solutions
    for col in range(N):
        if is_safe(solution, row, col):
            solution.append(col)
            backtrack_basic(row+1, solution, solutions)
            solution.pop()
    return solutions

# --- Chiến lược 2: MRV (chọn hàng có ít cột khả thi nhất) ---
def backtrack_mrv(solution=None, solutions=None):
    if solution is None:
        solution, solutions = [], []
    if len(solution) == N:
        solutions.append(solution[:])
        return solutions

    row = len(solution)
    domain = [c for c in range(N) if is_safe(solution, row, c)]

    if not domain:
        return solutions

    for col in domain:
        new_sol = solution[:] + [col]
        backtrack_mrv(new_sol, solutions)
    return solutions

# --- Chiến lược 3: LCV (chọn giá trị gây ít xung đột nhất) ---
def order_lcv(solution, row):
    def conflicts(col):
        count = 0
        for r in range(row+1, N):
            for c in range(N):
                if not is_safe(solution + [col], r, c):
                    count += 1
        return count
    return sorted(range(N), key=conflicts)

def backtrack_lcv(row=0, solution=None, solutions=None):
    if solution is None:
        solution, solutions = [], []
    if row == N:
        solutions.append(solution[:])
        return solutions
    for col in order_lcv(solution, row):
        if is_safe(solution, row, col):
            solution.append(col)
            backtrack_lcv(row+1, solution, solutions)
            solution.pop()
    return solutions

# --- Hàm chạy thử nghiệm ---
def run_solver(solver, name):
    start = time.time()
    sols = solver()
    end = time.time()
    print(f"{name}: {len(sols)} solutions found in {end-start:.4f}s")

    # In thử tối đa 3 nghiệm đầu
    for i, sol in enumerate(sols[:3], 1):
        print(f"Solution {i}: {sol}")
        print_board(sol)

# Thử nghiệm với các chiến lược
run_solver(backtrack_basic, "Basic Backtracking")
run_solver(backtrack_mrv,   "MRV")
run_solver(backtrack_lcv,   "LCV")


Basic Backtracking: 10 solutions found in 0.0002s
Solution 1: [0, 2, 4, 1, 3]
 Q  .  .  .  . 
 .  .  Q  .  . 
 .  .  .  .  Q 
 .  Q  .  .  . 
 .  .  .  Q  . 
---------------
Solution 2: [0, 3, 1, 4, 2]
 Q  .  .  .  . 
 .  .  .  Q  . 
 .  Q  .  .  . 
 .  .  .  .  Q 
 .  .  Q  .  . 
---------------
Solution 3: [1, 3, 0, 2, 4]
 .  Q  .  .  . 
 .  .  .  Q  . 
 Q  .  .  .  . 
 .  .  Q  .  . 
 .  .  .  .  Q 
---------------
MRV: 10 solutions found in 0.0002s
Solution 1: [0, 2, 4, 1, 3]
 Q  .  .  .  . 
 .  .  Q  .  . 
 .  .  .  .  Q 
 .  Q  .  .  . 
 .  .  .  Q  . 
---------------
Solution 2: [0, 3, 1, 4, 2]
 Q  .  .  .  . 
 .  .  .  Q  . 
 .  Q  .  .  . 
 .  .  .  .  Q 
 .  .  Q  .  . 
---------------
Solution 3: [1, 3, 0, 2, 4]
 .  Q  .  .  . 
 .  .  .  Q  . 
 Q  .  .  .  . 
 .  .  Q  .  . 
 .  .  .  .  Q 
---------------
LCV: 10 solutions found in 0.0015s
Solution 1: [0, 3, 1, 4, 2]
 Q  .  .  .  . 
 .  .  .  Q  . 
 .  Q  .  .  . 
 .  .  .  .  Q 
 .  .  Q  .  . 
---------------
Solution 2: 

# YC3

In [1]:
import time
from simpleai.search import CspProblem, backtrack
from collections import deque

# --- Đếm số lần kiểm tra ràng buộc ---
constraint_calls = 0

def queens_constraint(variables, values):
    """
    Ràng buộc N-Queens: không cùng cột và không cùng đường chéo.
    """
    global constraint_calls
    constraint_calls += 1
    for i in range(len(values)):
        for j in range(i + 1, len(values)):
            if values[i] == values[j] or abs(i - j) == abs(values[i] - values[j]):
                return False
    return True

# --- AC3 cho simpleai ---
def ac3_simpleai(problem):
    queue = deque((xi, xj) for xi in problem.variables for xj in problem.variables if xi != xj)
    while queue:
        xi, xj = queue.popleft()
        if revise(problem, xi, xj):
            if not problem.domains[xi]:
                return False
            for xk in problem.variables:
                if xk != xi and xk != xj:
                    queue.append((xk, xi))
    return True

def revise(problem, xi, xj):
    revised = False
    to_remove = set()
    for x in problem.domains[xi]:
        # Kiểm tra xem có giá trị nào ở xj thỏa mãn ràng buộc không
        if not any(queens_constraint([xi, xj], [x, y]) for y in problem.domains[xj]):
            to_remove.add(x)
            revised = True
    problem.domains[xi] = [v for v in problem.domains[xi] if v not in to_remove]
    return revised

# --- Hàm giải N-Queens ---
def solve_nqueens(n, use_ac3=False):
    global constraint_calls
    constraint_calls = 0
    variables = list(range(n))
    domains = {v: list(range(n)) for v in variables}
    constraints = [(variables, queens_constraint)]
    problem = CspProblem(variables, domains, constraints)

    if use_ac3:
        ac3_simpleai(problem)

    start = time.time()
    result = backtrack(problem)
    end = time.time()
    return result, end - start, constraint_calls

# --- Hàm in bàn cờ ---
def print_board(solution, n):
    for r in range(n):
        row = ''
        for c in range(n):
            row += 'Q ' if solution[r] == c else '. '
        print(row)

# --- Main ---
if __name__ == "__main__":
    print("--- N-Queens (N=5) CSP: So sánh Backtracking và Backtracking + AC3 ---")
    for ac3_flag in [False, True]:
        result, t, steps = solve_nqueens(5, use_ac3=ac3_flag)
        print(f"\nAC3 = {ac3_flag}: {result}")
        print(f"Time: {t:.4f}s, Constraint calls: {steps}")
        print("Bàn cờ:")
        print_board(result, 5)


--- N-Queens (N=5) CSP: So sánh Backtracking và Backtracking + AC3 ---

AC3 = False: {0: 0, 1: 2, 2: 4, 3: 1, 4: 3}
Time: 0.0076s, Constraint calls: 359
Bàn cờ:
Q . . . . 
. . Q . . 
. . . . Q 
. Q . . . 
. . . Q . 

AC3 = True: {0: 0, 1: 2, 2: 4, 3: 1, 4: 3}
Time: 0.0050s, Constraint calls: 559
Bàn cờ:
Q . . . . 
. . Q . . 
. . . . Q 
. Q . . . 
. . . Q . 


# YC4

In [ ]:
import random
import time
import pandas as pd
from simpleai.search import SearchProblem
from simpleai.search.local import hill_climbing, simulated_annealing, genetic


class NQueensProblem(SearchProblem):
    def __init__(self, n):
        self.n = n
        initial = tuple(random.randrange(n) for _ in range(n))
        super().__init__(initial_state=initial)

    def actions(self, state):
        actions = []
        for row in range(self.n):
            for col in range(self.n):
                if col != state[row]:
                    actions.append((row, col))
        return actions

    def result(self, state, action):
        row, col = action
        new_state = list(state)
        new_state[row] = col
        return tuple(new_state)

    def value(self, state):
        # Maximize negative conflicts
        return -self.conflicts(state)

    def conflicts(self, state):
        n = self.n
        count = 0
        for r1 in range(n):
            for r2 in range(r1 + 1, n):
                c1, c2 = state[r1], state[r2]
                if c1 == c2 or abs(c1 - c2) == abs(r1 - r2):
                    count += 1
        return count

    def generate_random_state(self):
        """Required for genetic algorithm in simpleai"""
        return tuple(random.randrange(self.n) for _ in range(self.n))
    
    def crossover(self, state1, state2):
        """Single-point crossover between two boards"""
        cut = random.randint(0, self.n - 1)
        child = state1[:cut] + state2[cut:]
        return tuple(child)

    def mutate(self, state):
        """Mutate by moving one queen to a random column"""
        row = random.randrange(self.n)
        col = random.randrange(self.n)
        new_state = list(state)
        new_state[row] = col
        return tuple(new_state)


# Wrappers
def hill_climbing_ai(n=5, max_steps=1000):
    problem = NQueensProblem(n)
    result = hill_climbing(problem, iterations_limit=max_steps)
    return list(result.state)


def simulated_annealing_ai(n=5, max_steps=1000):
    problem = NQueensProblem(n)
    result = simulated_annealing(problem, iterations_limit=max_steps)
    return list(result.state)


def genetic_ai(n=5, pop_size=50, generations=500, mutation_rate=0.2):
    problem = NQueensProblem(n)
    result = genetic(
        problem,
        iterations_limit=generations,
        population_size=pop_size,
        mutation_chance=mutation_rate
    )
    return list(result.state)


# Experiment & Comparison
def compare_algorithms(n=5, trials=5):
    results = []

    for algo_name, algo in [
        ("Hill Climbing", hill_climbing_ai),
        ("Simulated Annealing", simulated_annealing_ai),
        ("Genetic Algorithm", genetic_ai),
    ]:
        for t in range(trials):
            start = time.time()
            solution = algo(n)
            elapsed = time.time() - start

            problem = NQueensProblem(n)
            conf = problem.conflicts(solution)

            results.append({
                "Algorithm": algo_name,
                "Trial": t + 1,
                "Conflicts": conf,
                "Solved": conf == 0,
                "Time (s)": round(elapsed, 4)
            })

    df = pd.DataFrame(results)
    return df



df = compare_algorithms(n=5, trials=100)
print(df)
print("\nSummary:")
print(df.groupby("Algorithm")[["Solved", "Conflicts", "Time (s)"]].mean())


             Algorithm  Trial  Conflicts  Solved  Time (s)
0        Hill Climbing      1          1   False    0.0003
1        Hill Climbing      2          1   False    0.0002
2        Hill Climbing      3          0    True    0.0005
3        Hill Climbing      4          0    True    0.0005
4        Hill Climbing      5          0    True    0.0004
..                 ...    ...        ...     ...       ...
295  Genetic Algorithm     96          5   False    0.3273
296  Genetic Algorithm     97          5   False    0.3961
297  Genetic Algorithm     98          3   False    0.3081
298  Genetic Algorithm     99          4   False    0.3110
299  Genetic Algorithm    100          4   False    0.3124

[300 rows x 5 columns]

Summary:
                     Solved  Conflicts  Time (s)
Algorithm                                       
Genetic Algorithm      0.00       4.25  0.323890
Hill Climbing          0.69       0.46  0.000356
Simulated Annealing    0.99       0.01  0.095937
